# Complaint Priority Classifier

This notebook uses a local Ollama model (`qwen2.5:3b-instruct`) to classify complaints into High, Medium, or Low priority.

In [1]:
import requests
import json

# Configuration
OLLAMA_URL = "http://localhost:11434/api/chat"
MODEL_NAME = "qwen2.5:3b-instruct"

# System Prompt Definition
SYSTEM_PROMPT = """
You are NOT a chatbot.
You are NOT an assistant.
You are a strict text classification model.

Your ONLY task is to analyze the complaint_text and return exactly ONE word from the following list:

High
Medium
Low

You must follow these rules strictly:
- Output ONLY one word.
- Do NOT explain.
- Do NOT generate code.
- Do NOT provide reasoning.
- Do NOT repeat the complaint.
- Do NOT add punctuation.
- Do NOT add extra words.
- Do NOT say anything before or after the answer.

Classification Rules:

High:
- Essential services affected (water, electricity, hospital access, sewage overflow)
- Public safety risk
- Health hazards
- Road completely blocked
- Fire, flooding, major damage
- Large number of people affected
- Emergency situations

Medium:
- Local inconvenience
- Parking issues
- Noise complaints
- Minor road damage
- Streetlight not working
- Garbage collection delay in small area
- Issues affecting limited number of people

Low:
- Personal disputes
- Very minor cleanliness issues
- Small inconvenience affecting very few people
- Non-urgent or informational complaints

Examples:

complaint_text = "no water supply today in my area"
Output: High

complaint_text = "unknown car parking in front of my home"
Output: Medium

complaint_text = "neighbor playing loud music during afternoon"
Output: Medium

complaint_text = "small crack in pavement near my gate"
Output: Low
"""


In [2]:
def classify_complaint(complaint_text):
    """
    Classifies the complaint text using the local Ollama model.
    """
    payload = {
        "model": MODEL_NAME,
        "messages": [
            { "role": "system", "content": SYSTEM_PROMPT },
            { "role": "user", "content": f"complaint_text = \"{complaint_text}\"" }
        ],
        "stream": False,
        "options": {
            "temperature": 0.0  # Deterministic output
        }
    }
    
    try:
        response = requests.post(OLLAMA_URL, json=payload)
        response.raise_for_status()
        # The response structure depends on the endpoint. For /api/chat it's message.content
        # For /api/generate it's response
        result = response.json()
        # Ollama /api/chat response: { "model": "...", "created_at": "...", "message": { "role": "assistant", "content": "..." }, ... }
        if "message" in result:
            priority = result["message"]["content"].strip()
        else:
            # Fallback if using /api/generate structure (unlikely given URL but safe to check)
            priority = result.get("response", "Error: Unexpected response format").strip()
            
        return priority
    except requests.exceptions.RequestException as e:
        return f"Error: {e}"


In [3]:
# Example Usage
examples = [
    "The main water pipe burst and the street is flooding!",
    "There is a pothole on 5th avenue.",
    "My neighbor's dog barks at night."
]

print(f"Testing Model: {MODEL_NAME}\n")

for complaint in examples:
    print(f"Complaint: {complaint}")
    priority = classify_complaint(complaint)
    print(f"Priority: {priority}\n")


Testing Model: qwen2.5:3b-instruct

Complaint: The main water pipe burst and the street is flooding!
Priority: High

Complaint: There is a pothole on 5th avenue.
Priority: Low

Complaint: My neighbor's dog barks at night.
Priority: Medium



In [ ]:
# Interactive Loop
print("Complaint Priority Classifier (Type 'exit' to quit)")
print("-" * 50)

while True:
    user_input = input("\nEnter complaint text: ")
    if user_input.lower() in ['exit', 'quit']:
        break
    
    if not user_input.strip():
        continue
        
    priority = classify_complaint(user_input)
    print(f"Priority: {priority}")


Complaint Priority Classifier (Type 'exit' to quit)
--------------------------------------------------



Enter complaint text:  neighbour broke handle of my car.


Priority: Low



Enter complaint text:  there is a cou ding in front of my house


Priority: Low



Enter complaint text:  modiji slapped me.


Priority: Low
